In [2]:
# IMPORTS
import pandas as pd
import numpy as np

L’identification des acteurs les plus présents et les périodes associées

Dans la table title.principals.tsv.gz on récupère toutes les lignes category == 'actor' 

On récupère de l'autre côté la table title.basics.tsv.gz avec que les films

BASES A CHARGER

In [3]:
# On charge la bd title.principals
df_acteur = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
df_acteur = df_acteur[(df_acteur['category'] == 'actor') | (df_acteur['category'] == 'actress') | (df_acteur['category'] == 'self')]

# On charge la bd title.basics

df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_film = df_film[df_film['titleType'] == 'movie']

# On charge la bd name.basics
 
df_nom = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')

'''
df_acteur -> table des acteurs présents dans un film
df_film -> table des films
df_nom -> table des noms des acteurs
'''

MERGES

In [9]:
# MERGE Acteurs et Films INNER

df_acteur_film_inner = pd.merge(df_acteur,
                                df_film,
                                how = 'inner',
                                on = 'tconst') 

# MERGE Acteurs et Films OUTER

df_acteur_film_outer = pd.merge(df_acteur,
                                df_film,
                                how = 'outer',
                                on = 'tconst') 

Nombre d'apparitions

In [14]:
# On compte le nombre d'occurences de chaque acteur dans ce nouveau df

df_vc = df_acteur_film_inner['nconst'].value_counts()

In [15]:
# On charge le df dans lequel le nom des acteurs est contenu
# et on le merge avec notre df du nombre d'occurrences pour
# obtenir un acteur et son nb d'occurence

df_apparition = pd.merge(df_vc,
                          df_nom,
                          how = 'left',
                          on = 'nconst') 

MIN et MAX de STARTYEAR

In [16]:
# On supprime les lignes où StartYear est vide ou est égal à \N

df_acteur_film_outer = df_acteur_film_outer[~(df_acteur_film_outer['startYear'] == '\\N')]
df_acteur_film_outer = df_acteur_film_outer[~(df_acteur_film_outer['startYear'].isna())]

# On supprime les lignes où NCONST est vide

df_acteur_film_outer = df_acteur_film_outer[~(df_acteur_film_outer['nconst'].isna())]

In [18]:
# On crée deux colonnes: une avec l'année minimum et l'autre max

df_min = df_acteur_film_outer.groupby('nconst')['startYear'].min()
df_max = df_acteur_film_outer.groupby('nconst')['startYear'].max()

In [19]:
# On merge les dates MIN et MAX de StartYear

df_dates = pd.merge(df_max,
                    df_min,
                    how = 'outer',
                    on = 'nconst') 

In [38]:
# On merge df_apparition avec le Min de StartYear et le Max de StartYear

df_kpi1 = pd.merge(df_apparition,
                   df_dates,
                   how = 'left',
                   on = 'nconst') 

In [39]:
# On rename des colonnes pour y voir clair

df_kpi1 = df_kpi1.rename(columns = {'startYear_y' : 'Premier film','startYear_x' : 'Dernier film'})

# On garde uniquement les colonnes nécessaires

df_kpi1 = df_kpi1[['primaryName', 'count', 'Premier film', 'Dernier film']]

# On rename les colonnes conservées

df_kpi1 = df_kpi1.rename(columns = {'primaryName' : 'Acteur/Actrice','count' : 'Nombre de films'})

In [40]:
# Résultat final

df_kpi1

Acteur/Actrice  Nombre de films Premier film Dernier film
0             Brahmanandam             1131         1987         2025
1        Jagathy Sreekumar              661         1957         2023
2              Adoor Bhasi              626         1953         1991
3             Paquito Diaz              622         1958         2005
4             Eddie Garcia              616         1949         2019
...                    ...              ...          ...          ...
1445425  Kayleigh Lawrence                1         2009         2009
1445426      Chelsea Dixon                1         2009         2009
1445427        Luke Emsley                1         2009         2009
1445428       Teijun Ogawa                1         2008         2008
1445429      Bob Wolfenson                1         2013         2013

[1445430 rows x 4 columns]